In [5]:
%%capture

!pip install gdown gpxpy

In [3]:
%%capture

import gdown

DRIVE_FOLDER = 'https://drive.google.com/drive/folders/10Q8FkfqAhp8ShVB_AqwDvCrNzU69NFId?usp=sharing'
gdown.download_folder(DRIVE_FOLDER, quiet=True)

In [22]:
import gpxpy
import pandas as pd

def parse_gpx(filepath):
  gpx_file = open(filepath, 'r')
  gpx = gpxpy.parse(gpx_file)

  data = []

  for track in gpx.tracks:
    for segment in track.segments:
      for point in segment.points:
        time, lat, lng = point.time, point.latitude, point.longitude,
        data.append({
          'time': time,
          'lat': lat,
          'lon': lng,
        })

  df = pd.DataFrame(data)
  df['time'] = pd.to_datetime(df['time'])

  return df

In [23]:
osmtracker_df = parse_gpx('/content/polarflow-vs-osmtracker/osmtracker.gpx')
osmtracker_df.head()

,time,lat,lon
0,2024-11-16 23:52:21+00:00,20.994104,105.869489
1,2024-11-16 23:52:22+00:00,20.994249,105.869443
2,2024-11-16 23:52:24+00:00,20.993869,105.869319
3,2024-11-16 23:52:25+00:00,20.993885,105.869305
4,2024-11-16 23:52:26+00:00,20.993910,105.869322


In [24]:
polarflow_df = parse_gpx('/content/polarflow-vs-osmtracker/polarflow.gpx')
polarflow_df.head()

,time,lat,lon
0,2024-11-16 23:51:44.004000+00:00,20.993463,105.869606
1,2024-11-16 23:51:45.001000+00:00,20.993463,105.869606
2,2024-11-16 23:51:46.001000+00:00,20.993476,105.869620
3,2024-11-16 23:51:47.001000+00:00,20.993491,105.869634
4,2024-11-16 23:51:48.001000+00:00,20.993512,105.869643


In [15]:
import folium

m = folium.Map(
    location=(
      osmtracker_df['lat'].mean(),
      osmtracker_df['lon'].mean(),
    ),
    zoom_start=14,
)

def add_to_map(df, color):
    coordinates = df[['lat', 'lon']].values.tolist()
    folium.PolyLine(locations=coordinates, color=color, weight=5, opacity=1).add_to(m)

add_to_map(osmtracker_df, 'black')
add_to_map(polarflow_df, 'orange')
m

In [17]:
osmtracker_df.shape

(2641, 2)

In [18]:
polarflow_df.shape

(2753, 2)

In [28]:
import numpy as np

def get_time_deltas(df):
    df_sorted = df.sort_values(by='time')

    time_deltas = df_sorted['time'].diff().dropna()
    time_deltas_seconds = time_deltas.dt.total_seconds()

    p10 = np.percentile(time_deltas_seconds, 10)
    p50 = np.percentile(time_deltas_seconds, 50)
    p90 = np.percentile(time_deltas_seconds, 90)
    p99 = np.percentile(time_deltas_seconds, 99)

    print(f"P10: {p10}")
    print(f"P50: {p50}")
    print(f"P90: {p90}")
    print(f"P99: {p99}")

In [29]:
get_time_deltas(osmtracker_df)

P10: 1.0
P50: 1.0
P90: 1.0
P99: 2.0


In [30]:
get_time_deltas(polarflow_df)

P10: 0.999
P50: 1.0
P90: 1.001
P99: 1.006
